In [1]:
using AcausalNets
using LightGraphs
using QuantumInformation
using LinearAlgebra

┌ Info: Recompiling stale cache file /home/marcin/.julia/compiled/v0.7/AcausalNets/sM5bh.ji for AcausalNets [f45f996a-9c6d-58c4-b63c-9e6370055198]
└ @ Base loading.jl:1185
┌ Warning: Package AcausalNets does not have QuantumInformation in its dependencies:
│ - If you have AcausalNets checked out for development and have
│   added QuantumInformation as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AcausalNets
└ Loading QuantumInformation into AcausalNets from project dependency, future warnings for AcausalNets are suppressed.


In [2]:
roA = Diagonal([1/3, 1/3, 1/3])

roB = Diagonal([1/3, 1/3, 1/3])
roCwAB = Diagonal([
        0,1/2, 1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        #CHANGED to that ro(C | AB) != ro(C | BA) - Originally BE 0,1,0
        0,1/2,0, #A=0, B=2 
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1
        1/2,1/2,0 #A=2, B=2
        ]); #


var_a = Variable(:a, 3)

var_b = Variable(:b, 3)
var_c = Variable(:c, 3)

sys_a = DiscreteQuantumSystem([var_a], roA)

sys_b = DiscreteQuantumSystem([var_b], roB)

sys_c_ab = DiscreteQuantumSystem([var_a, var_b], [var_c], roCwAB)
an = AcausalNet()

push!(an, sys_a)
push!(an, sys_b)
push!(an, sys_c_ab)
# show(an)

AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}({3, 2} directed simple Int64 graph, AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}[DiscreteSystem{AbstractArray{T,2} where T}(Variable[], Variable[Variable(:a, 3)], [0.333333 0.0 0.0; 0.0 0.333333 0.0; 0.0 0.0 0.333333]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[], Variable[Variable(:b, 3)], [0.333333 0.0 0.0; 0.0 0.333333 0.0; 0.0 0.0 0.333333]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 3), Variable(:b, 3)], Variable[Variable(:c, 3)], [0.0 0.0 … 0.0 0.0; 0.0 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.5 0.0; 0.0 0.0 … 0.0 0.0])])

In [3]:
is_parent(sys_a, sys_c_ab)

true

In [4]:
# the original matrix is ro(C | AB)
# this is ro(C | BA)
roCwBA = permute_systems(roCwAB, [3,3,3], [2, 1, 3])
print(roCwBA == roCwAB)
sys_c_ba = DiscreteQuantumSystem([var_b, var_a], [var_c], roCwBA)

# joint system (a, a2) where a and a2 are independent
# ncategories a = 3
# ncategories a2 = 2

roAA2 = Diagonal([1/6, 1/6, 1/6, 1/6, 1/6, 1/6])
var_a2 = Variable(:a2, 2)
sys_aa2 = DiscreteQuantumSystem([var_a, var_a2], roAA2)

an_2 = AcausalNet()
push!(an_2, sys_aa2)
push!(an_2, sys_b)

# we push a system with roCwBA
# during push, it's parents will be expanded to include var_a2 
# (since it's in joint state with var_a)
# and the parents will be permuted to bein the same order 
# as the BayesNet's order of variables
# before ppermutation their order is: a2, b, a
# after permutation their order is: a, a2, b
push!(an_2, sys_c_ba)
# show(an_2)

false

AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}({3, 2} directed simple Int64 graph, AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}[DiscreteSystem{AbstractArray{T,2} where T}(Variable[], Variable[Variable(:a, 3), Variable(:a2, 2)], [0.166667 0.0 … 0.0 0.0; 0.0 0.166667 … 0.0 0.0; … ; 0.0 0.0 … 0.166667 0.0; 0.0 0.0 … 0.0 0.166667]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[], Variable[Variable(:b, 3)], [0.333333 0.0 0.0; 0.0 0.333333 0.0; 0.0 0.0 0.333333]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 3), Variable(:a2, 2), Variable(:b, 3)], Variable[Variable(:c, 3)], [0.0 0.0 … 0.0 0.0; 0.0 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.5 0.0; 0.0 0.0 … 0.0 0.0])])

In [5]:
is_parent(sys_a, sys_c_ba)

true

In [6]:
is_parent(sys_aa2, sys_c_ba)

LoadError: Only some of the variables in parent are listed as child's parents.
                Inputting the systems into an AcausalNet may fix that problem.

In [7]:
c_system = an_2.systems[variable_to_node(var_c, an_2)]
roC_AA2B = c_system.distribution

traced_out = ptrace(roC_AA2B, [3,2,3,3], [2]) / var_a2.ncategories

traced_out == roCwAB

true

In [8]:
is_parent(sys_aa2, c_system)

true

In [15]:
import AcausalNets.Systems: DiscreteSystem, QuantumDistribution
import AcausalNets.Algebra: eye
import AcausalNets.Common: is_subset

In [10]:
identity_distribution(::Type{D}, size::Int64) where {D <: QuantumDistribution} = eye(size)

identity_distribution (generic function with 1 method)

In [11]:
identity_distribution(QuantumDistribution, 5)

5×5 Diagonal{Float64,Array{Float64,1}}:
 1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅   1.0

In [14]:
methods(relevant_variables)

# 1 method for generic function "relevant_variables":
[1] relevant_variables(system::DiscreteSystem) in AcausalNets.Systems at /home/marcin/.julia/dev/AcausalNets/src/Systems/discrete_system.jl:35

In [21]:
is_subset(Set([1,2]), Set([1,2,3]))

true

In [12]:
function merge_systems(systems::Vector{DiscreteSystem{D}})::DiscreteSystem{D} where D
    all_variables = vcat([variables(s) for s in systems])
    for sys_1 in systems
        mul_elem = identity_distribution(D, 1)
        multiplied_indices = []
        multiplied_dimensions = []
        for sys_2 in systems
            factor = identity_distribution(D, 1)
            if sys_1 == sys_2 && 
                is_subset(Set(relevant_variables(sys)), Set(all_variables)) 
                assignment_factor = identity_distribution(D, 1)
                
            end
                
        end
    end
end

merge_systems (generic function with 1 method)